## 7 畳み込みニューラルネットワーク（CNN）

## 7.2 im2colとcol2im

### 7.2.3 im2colの実装 -シンプルなim2col-

In [1]:
import numpy as np

def im2col(image, flt_h, flt_w, out_h, out_w): #1
    
    img_h, img_w = image.shape # 入力画像の高さ、幅
    
    # 生成される行列のサイズ
    cols = np.zeros((flt_h * flt_w, out_h * out_w))
    
    for h in range(out_h): 
        h_lim = h + flt_h
        for w in range(out_w):
            w_lim = w + flt_w
            cols[:, h * out_w + w] = image[h:h_lim, w:w_lim].reshape(-1)
            
    return cols

In [2]:
img = np.array([[1, 2, 3, 4], 
                [5, 6, 7, 8], 
                [9, 10, 11, 12], 
                [13, 14, 15, 16]])
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


In [3]:
import numpy as np

def im2col(image, flt_h, flt_w, out_h, out_w):
    
    img_h, img_w = image.shape # 入力画像の高さ、幅
    
    # 生成される行列のサイズ
    cols = np.zeros((flt_h, flt_w, out_h, out_w))
    
    for h in range(flt_h): 
        h_lim = h + out_h
        for w in range(flt_w):
            w_lim = w + out_w
            cols[h, w, :, :] = image[h:h_lim, w:w_lim]
    
    cols = cols.reshape(flt_h*flt_w, out_h*out_w)
            
    return cols

In [4]:
img = np.array([[1, 2, 3, 4], 
                [5, 6, 7, 8], 
                [9, 10, 11, 12], 
                [13, 14, 15, 16]])
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


### 7.2.4 im2colの実装 -im2colの実用化

In [5]:
# バッチとチャンネルに対応
def im2col(images, flt_h, flt_w, out_h, out_w):
   
    n_bt, n_ch, img_h, img_w = images.shape # バッチサイズ、チャンネル数、入力画像高さ、幅
    
    cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))

    for h in range(flt_h):
        h_lim = h + out_h
        for w in range(flt_w):
            w_lim = w + out_w
            cols[:, :, h, w, :, :] = images[:, :, h:h_lim, w:w_lim]

    cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(n_ch*flt_h*flt_w, n_bt*out_h*out_w)
    return cols

In [6]:
img = np.array([[[[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10,11,12],
                  [13,14,15,16]]]])
cols = im2col(img, 2, 2, 3, 3)


バッチとチャンネルに対応しパディングとストライドを考慮したim2col

In [7]:
# coding: UTF-8
#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
def im2col(images, flt_h, flt_w, out_h, out_w, stride, pad):
   
    n_bt, n_ch, img_h, img_w = images.shape
    
    img_pad = np.pad(images, [(0,0), (0,0), (pad, pad), (pad, pad)], "constant")
    cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))

    for h in range(flt_h):
        h_lim = h + stride*out_h
        for w in range(flt_w):
            w_lim = w + stride*out_w
            cols[:, :, h, w, :, :] = img_pad[:, :, h:h_lim:stride, w:w_lim:stride]

    cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(n_ch*flt_h*flt_w, n_bt*out_h*out_w)
    return cols

#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
#1 image:入力画像，
#  flt_h:フィルタの高さ，flt_w:フィルタの幅, out_h:出力画像の高さ, out_w:出力画像の幅

In [8]:
img = np.array([[[[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10,11,12],
                  [13,14,15,16]]]])
cols = im2col(img, 2, 2, 3, 3, 1, 0)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


### 7.2.6 col2im の実装

In [9]:
# coding: UTF-8
#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
def col2im(cols, img_shape, flt_h, flt_w, out_h, out_w, stride, pad):
 
    n_bt, n_ch, img_h, img_w = img_shape
    
    # 6次元配列に分解して軸を入れ替えて，配列形状は (B, C, Fh, Fw, Oh, Ow) となる．
    cols = cols.reshape(n_ch, flt_h, flt_w, n_bt, 
                        out_h, out_w).transpose(3, 0, 1, 2, 4, 5) 
    
    # 変換後の画像を格納する4次元配列の作成
    images = np.zeros((n_bt, n_ch, img_h + 2*pad + stride - 1,
                       img_w + 2 * pad + stride -1)) 
    # stride-1を加えるのは画像がストライドの値で割り切れない場合の対策
    
    for h in range(flt_h):
        h_lim = h + stride*out_h
        for w in range(flt_w):
            w_lim = w + stride*out_w
            images[:, :, h:h_lim:stride, w:w_lim:stride] += cols[:, :, h, w, :, :]
            # imagesにcolsにおけるフィルタの該当箇所を格納する

    return images[:, :, pad:img_h+pad, pad:img_w+pad]
            # パディングの分をスライスによって取り除いている

#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
#1 cols:引数として受け取る行列
#  受け取るcolsの形状は (n_ch * flt_h * flt_w, n_bt * out_h * ouy_w)
#  n_ch:チャンネル数，n_bt:バッチサイズ数,

In [10]:
cols = np.ones((4, 4))
img_shape = (1, 1, 3, 3)
images = col2im(cols, img_shape, 2, 2, 2, 2, 1, 0)
print(images)

[[[[1. 2. 1.]
   [2. 4. 2.]
   [1. 2. 1.]]]]
